# モンテカルロ法
DPを用いて最適評価関数と最適方策を得る手法は環境のモデルが既知である必要がある。

モンテカルロ法は、未知の環境においてデータのサンプリングを行うことで推定を行う手法の総称である。

## 分布モデルとサンプルモデル
**分布モデル**は確率分布として表されたモデルを指す。

| さいころの目の和 | 2 | 3 | 4 | 5 | 6 | 7 | 8 | 9 | 10 | 11 | 12 |
| :----: | :----: | :----: | :----: | :----: | :----: | :----: | :----: | :----: | :----: | :----: | :----: |
| 確率 | $\frac 1 {36}$ | $\frac 2 {36}$ | $\frac 3 {36}$ | $\frac 4 {36}$ | $\frac 5 {36}$ | $\frac 6 {36}$ | $\frac 5{36}$ | $\frac 4 {36}$ | $\frac 3 {36}$ | $\frac 2 {36}$ | $\frac 1 {36}$ |

一方、**サンプルモデル**はサンプリングができるモデルを指す。

## 分布モデルにおける期待値の算出
分布モデルでは確率分布があらかじめわかっているため、以下のように算出できる。

In [1]:
p = {2: 1/36, 3: 2/36, 4: 3/36, 5: 4/36, 6: 5/36, 7: 6/36, 8: 5/36, 9: 4/36, 10: 3/36, 11: 2/36, 12: 1/36}

V = sum(k * v for k, v in p.items())
print(V)

7.0


## サンプルモデルにおける期待値の算出
十分な数のサンプルを使うことで、期待値は正しい値に収束する。

十分な回数サンプルを取得し、計算することで推定する手法が**モンテカルロ法**である。

以下にモンテカルロ法で期待値を算出するプログラムを示す。真の期待値である$7$に近い値を得られていることがわかる。

In [2]:
import numpy as np

def sample(dices):
    x = 0
    for i in range(dices):
        x += np.random.randint(1, 7)
    return x

trial = 10000

samples = []
for i in range(trial):
    s = sample(2)
    samples.append(s)
    
V = sum(samples) / len(samples)
print(V)

6.9933


上記のプログラムではサンプルを全て求めてから期待値を計算している。

ここで、N個のデータから得られる平均値は以下のようにして求められる。

$$
V_n = \frac {s_1 + s_2 + ... + s_n} n
$$

これを変形すると以下のようになる。

$$
V_n = \frac {s_1 + s_2 + ... + s_n} n\\
V_n = \frac {s_n} n + \frac {n - 1} n \frac {s_1 + s_2 + ... + s_{n - 1}} {n - 1}\\
V_n = \frac {s_n} n + \biggl( 1 - \frac 1 n \biggr) V_{n - 1}\\
V_n = V_{n - 1} + \frac 1 n (s_n - V_{n - 1})\\
$$

よって、サンプルを得るたびにそれまでの平均値と新たに得たサンプルの値を用いて平均値を更新できる。

これを実装すると以下のようになる。

In [3]:
trial = 10000
V, n = 0, 0

for i in range(trial):
    s = sample(2)
    n += 1
    V += (s - V) / n
print(V)

7.043499999999998


## 価値関数をモンテカルロ法で求める
まず、価値関数は以下の式で表される。

$$
u_\pi (s) = \mathbb{E} [G|s]
$$

ここにモンテカルロ法を適用することを考えると、$n$回目の試行で得た収益を$G^{(n)}$として以下の式で表すことができる。なお、この際に方策$\pi$を用いる。

$$
V_\pi(s) = \frac {G^{(1)} + G^{(2)} + ... + G^{(n)}} n
$$

これを各状態について求めればよい。このとき、各状態の価値関数の値どうしは独立して算出されていることに注意が必要である。